In [1]:
import numpy as np
import pandas as pd
import re
import pickle
from unicodedata import normalize
import string
import keras
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import Callback
import warnings
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Bidirectional, LSTM, Dense, Concatenate
from tensorflow.keras.initializers import Constant
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from datasets import load_dataset
import collections
import json
from google.transliteration import transliterate_text

/Users/haseeb/miniconda3/envs/English2Hinglish_tf/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
import os
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
#using this implementation of attention
#https://github.com/thushv89/attention_keras/blob/master/src/layers/attention.py
class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def _init_(self, **kwargs):
        super(AttentionLayer, self)._init_(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [3]:
loaded_model = keras.models.load_model("Seq2seq_transformer_model.h5",custom_objects={"AttentionLayer": AttentionLayer})

In [4]:
#encoder inference model
encoder_inf_inputs = loaded_model.input[0]  # input_1
encoder_inf_output2,encoder_inf_state_h2, encoder_inf_state_c2 = loaded_model.layers[5].output
encoder_inf_model = Model(inputs=encoder_inf_inputs,outputs=[encoder_inf_output2,encoder_inf_state_h2, encoder_inf_state_c2])

In [5]:
# decoder inference model
latent_dim = 300
decoder_inf_inputs = loaded_model.input[1]

encoder_out_temp = Input(shape=(40,latent_dim), name = "encoder_input")
encoder_state_input_h = Input(shape=(latent_dim,), name = "encoder_hidden_state")
encoder_state_input_c = Input(shape=(latent_dim,), name = "encoder_cell_state")

decoder_embed_layer = loaded_model.layers[4]
decoder_embeddings = decoder_embed_layer(decoder_inf_inputs)

decoder_inf_lstm = loaded_model.layers[6]
decoder_inf_outputs, decoder_state_h2, decoder_state_c2 = decoder_inf_lstm(decoder_embeddings, initial_state=[encoder_state_input_h, encoder_state_input_c])

# attention_inf_layer = loaded_model.layers[7], this is the attention model from the model but 
decoder_attention = loaded_model.layers[7]
attn_out_inf,_=decoder_attention([encoder_out_temp, decoder_inf_outputs])
# concat
decoder_inf_concat=tf.keras.layers.Concatenate()([decoder_inf_outputs, attn_out_inf])
# time_distributed
time = loaded_model.layers[9]
decoder_outputs2 = time(decoder_inf_concat)

decoder_model = Model(
inputs = [decoder_inf_inputs] + [encoder_out_temp,encoder_state_input_h, encoder_state_input_c],
outputs = [decoder_outputs2] + [decoder_state_h2, decoder_state_c2])

In [7]:
with open("assets/en_dict.json", "r") as json_file:
    english_word_dict = json.load(json_file)

In [8]:
with open("assets/hing_rev_dict.json", "r") as json_file:
    hinglish_reversed_dict = json.load(json_file)

In [9]:
def prepare_text(text):
    # pad the text
    max_length = 40
    
    tokenized = word_tokenize(text)[:max_length]
    padded_text = tokenized + (max_length - len(tokenized)) * ["<padding>"]

    #initialising the training data
    encoder_input_data = np.zeros(
        (1, max_length),
        dtype='float32')
    
    for t, char in enumerate(padded_text):
        try:
            #use unk if we cant find the word in the dictionary
            encoder_input_data[0, t] = english_word_dict.get(
                char, english_word_dict["<unk>"])
        except:
            print(char)
    
    return encoder_input_data

In [10]:
sentence1 = "definitely share you feedback in the comment section"
sentence2 = "so even if its a big video, I will clearly mention all the products"
sentence3 = "I was waiting for my bag"

In [11]:
encoded_test = prepare_text(sentence1)

In [12]:
#encoding incoming data
e_out, e_h, e_c=encoder_inf_model.predict(encoded_test)
#creating target sequence
target_seq = np.zeros((1,1))
#inputing start token
target_seq[0, 0] = english_word_dict['<s>']
stop_condition = False
decoded_sentence = ''
#looping until stop token is predicted or 
while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
        [target_seq] + [e_out, e_h, e_c])
    #want to get index with highest probability
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    #find word at that index
    sampled_char = hinglish_reversed_dict[f"{sampled_token_index}"]
    print(sampled_char)
    #add to sentence
    decoded_sentence += ' '+sampled_char
    if (sampled_char == "</s>" or len(word_tokenize(decoded_sentence)) > 52):
        stop_condition = True
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index
    #update hidden + cell states
    e_h, e_c = h, c
# remove the 
decoded_sentence = decoded_sentence[:-4]

2023-11-06 20:02:04.879107: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 381ms/step
kya
1/1 [==============================] - 0s 22ms/step
aaj
1/1 [==============================] - 0s 21ms/step
raat
1/1 [==============================] - 0s 22ms/step
baarish
1/1 [==============================] - 0s 22ms/step
hogi
1/1 [==============================] - 0s 23ms/step
</s>


In [13]:
def transliterate(decoded_sentence):
    from google.transliteration import transliterate_text
    
    # Get english dictionary reference
    with open('assets/3kpop.txt', 'r') as file:
        file_content = file.read()
    wordlist = file_content.split()
    
    # transliterate
    hindi = transliterate_text(decoded_sentence, lang_code='hi')
    
    # check if words are english; if they are, replace them
    hinglish_final = hindi.split()
    for index,word in enumerate(decoded_sentence.split()):
        if word in wordlist:
            hinglish_final[index] = word
    #' '.join(hinglish_final)

    return ' '.join(hinglish_final)

In [14]:
print(decoded_sentence,'\n', transliterate(decoded_sentence))

 kya aaj raat baarish hogi  
 क्या आज रात बारिश होगी
